In [ ]:
using LazyGrids
using BlockArrays
using Printf
using SparseArrays
using SparseMatrixDicts
using SpecialFunctions
using FillArrays
using Parameters
using Test
using BenchmarkTools
using BasicInterpolators: BicubicInterpolator
using CairoMakie
using LaTeXStrings
CairoMakie.activate!()
using DelimitedFiles
using ColorSchemes
using JLD
using IterativeSolvers
# using Dierckx
using ScatteredInterpolation: interpolate, evaluate, InverseMultiquadratic
#using NaturalNeighbours: interpolate

In [ ]:
# file = load("GrowthRate_ep1_E1e-9.jld");
# k₁  = file["kₓ"];   
# λₛ₁  = file["λₛ"];
##
file = load("GrowthRate_ep1_E1e-9_3020_fdm.jld");
k₁  = file["kₓ"];   
λₛ₁  = file["λₛ"];
##
file = load("GrowthRate_ep10_E1e-9_3020_fdm.jld");
k₂  = file["kₓ"];   
λₛ₂  = file["λₛ"];
###

file = load("pertubarions_ep1_maxGrowthRate.jld");
y  = file["y"];   
z  = file["z"];
X  = file["X"]
ny = length(y)
nz = length(z)
N  = ny * nz
w_re  = reshape( real(X[1:1N]), (nz, ny) )

y1 = zeros(ny+1)
y1[1:ny] = y; 
y1[ny+1] = y1[ny] + y[2]-y[1]

w  = zeros(nz,ny+1)
w[:,1:ny] = w_re
w[:,ny+1] = w[:,1]

println(length(y))
println(length(z))

max_val = maximum(real(λₛ₁))
id = 0
for it in 1:length(k₁)
    if λₛ₁[it].re == max_val
        id = it
    end
end
println(λₛ₁[id])
k₁[id]

In [ ]:
##theoretical estimate (reproducing fig.1 of Stone 1971, JFM)
Ri  = 2.0  #same as in the `Params' struct

δ   = 1.0  #same as in the `Params' struct
kₜₕ₁ = range(1e-3, maximum(k₁), 200) |> collect
λₜₕ₁ = @. 0.5/√3 * ( kₜₕ₁ - 2kₜₕ₁^3/15*(1 + Ri + 5*δ^2*kₜₕ₁^2/42) );

δ   = 9 #same as in the `Params' struct
kₜₕ₂ = range(1e-3, maximum(k₂), 200) |> collect
λₜₕ₂ = @. 0.5/√3 * ( kₜₕ₂ - 2kₜₕ₂^3/15*(1 + Ri + 5*δ^2*kₜₕ₂^2/42) );

In [ ]:
fig  = Figure(fontsize=30, size = (1400, 600),  )

ax1   = Axis(fig[1,1], xlabel=L"$k$", xlabelsize=45,
        ylabel=L"$\sigma_i$", ylabelsize=45, 
        xgridvisible = false, ygridvisible = false, )
        
scatter!(k₁[1:end], real(λₛ₁[1:end]), markersize = 12, color = :black, label="numerical solution")
lines!(kₜₕ₁, λₜₕ₁, linewidth=3, color = :black, label="Stone (1971)")
# axislegend(position= :rt)
# xlims!(0., 1.1)
# ylims!(0, 0.2)
# hidespines!(ax1, :t, :r)

#ax2   = Axis(fig[1,2], xlabel=L"$k$", ylabel=L"$\sigma_r$")
scatter!(k₂, real(λₛ₂), markersize = 12, color = :blue) #, label="numerical solution")
lines!(kₜₕ₂, λₜₕ₂, linewidth=3, color = :blue) #, label="Stone (1971)")

text!(1.25, 0.15, text = L"$\delta=1$",  color=:black, fontsize=30)
text!(0.81, 0.09, text = L"$\delta=10$", color=:blue,  fontsize=30)

axislegend(framevisible=false,  position= :lt, fontsize=20)
xlims!(0., 2)
ylims!(0, 0.22)
hidespines!(ax1, :t, :r)

ax2 = Axis(fig[1,2], xlabel=L"$y$", xlabelsize=45,
        ylabel=L"$z$", ylabelsize=45, title=L"$w$",
        xgridvisible = false, ygridvisible = false, 
        aspect=1.1 )

max_val = maximum(abs.(w_re))
levels = range(-max_val, max_val, length=14)
co = contourf!(y1, z, w', colormap=cgrad(:balance, rev=false),
    levels=levels, extendlow = :auto, extendhigh = :auto )

xlims!(0., 1.0)
ylims!(0., 1.0)

#ax1.title = L"$\delta=1$"
#ax2.title = L"$w$"

Label(fig[1, 1, TopLeft()], L"$(a)$", fontsize=40, padding = (0, 0, 0, 0), halign = :right)
Label(fig[1, 2, TopLeft()], L"$(b)$", fontsize=40, padding = (0, 0, 0, 0), halign = :right)

display(fig)
#save("stone1971_benchmark.png", fig, px_per_unit=8)

In [ ]:
@inline λₜₕ(k, Γ, δ) = 1.0/2√3 * ( k - 2.0k^3/15*(1.0 + 1.0/Γ + 5.0δ^2*k^2/42) )
Γ = 0.5

k = 0.5496610169491526        #0.89
println(λₜₕ(k, Γ, 10.0))
0.5k